# PyTerrier Notebook for Full-Rank Submissions

This notebook serves as a baseline full-rank submission for [TIRA](https://tira.io)/[TIREx](https://tira.io/tirex) that builds a PyTerrier index and subsequently creates a run with BM25.

### Step 1: Ensure Libraries are Imported

In [ ]:
import os
import math

# Detect if we are in the TIRA sandbox
# Install the required dependencies if we are not in the sandbox.
if 'TIRA_DATASET_ID' not in os.environ:
    !pip3 install python-terrier tira==0.0.88 ir_datasets
else:
    print('We are in the TIRA sandbox.')

In [ ]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run

# this loads and starts pyterrier so that it also works in the TIRA
ensure_pyterrier_is_loaded()

# PyTerrier must be imported after the call to ensure_pyterrier_is_loaded in TIRA.
import pyterrier as pt
from pyterrier.measures import *

if not pt.started():
    pt.init(boot_packages=['mam10eks:custom-terrier-token-processing:0.0.1', 'com.github.terrierteam:terrier-prf:-SNAPSHOT'])
    from jnius import autoclass


### Step 2: Load the data

In [ ]:
data = pt.get_dataset('irds:ir-lab-jena-leipzig-wise-2023/validation-20231104-training')

### Step 3: Build the Index

### Step 4: Create the Retrieval Pipeline

#### Step 4.1: Add Query Expansion

### Step 5: Create the Run and Persist the Run

### Step 6: Run Experiments

In [ ]:
from tira import Client
tira_client = Client()

In [ ]:
from scipy.stats import ttest_rel

if 'TIRA_DATASET_ID' not in os.environ:
    # Führe das Experiment durch
    # PyTerrier führt standardmäßig nur two-sided ttest durch, daher müssen wir hier unseren eigenen one-sided ttest konfigurieren
    # PerQuery gibt für jede Query die eval_metrics zurück
    experiment = pt.Experiment(
        [
            # TODO: Load runs from TIRA
            pt.Transformer.from_df(
                tira_client.get_run_output(
                    '<task>/<team>/<approach>', '<dataset>')),
            pt.Transformer.from_df(
                tira_client.get_run_output(
                    '<task>/<team>/<approach>', '<dataset>')),
            pt.Transformer.from_df(
                tira_client.get_run_output(
                    '<task>/<team>/<approach>', '<dataset>')),
        ],
        data.get_topics(),
        data.get_qrels(),
        eval_metrics=["ndcg_cut_5"],
        names=[
            "BM25",
            "BM25 with linear weighting (ascending weight)",
            "BM25 with linear weighting (descending weight)",
        ],
        perquery=True
    )
    
    # Speicher alle Werte einer Pipeline in einem DF
    linear = experiment[experiment.name == "BM25 with linear weighting (ascending weight)"]
    inv_linear = experiment[experiment.name == "BM25 with linear weighting (descending weight)"]
    
    # Es werden nur die nDCG-Werte gebraucht
    inv_linear = inv_linear["value"]
    linear = linear["value"]
    
    # Das ist die unterliegende Test-Funktion, die auch von PyTerrier verwendet wird. Statt "two-sided" wird hier "greater" verwendet.
    print(ttest_rel(linear, inv_linear, alternative="greater").pvalue)

    # TODO: Ggf. Vergleich mit BM25 ohne weighting?